<a href="https://colab.research.google.com/github/qurainisepty/portfolio/blob/main/Titanic_survivor_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Library

In [ ]:
import pandas as pd
import numpy as np
import sklearn.feature_selection as fs
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

2. Data Collection


In [ ]:
data_mentah = pd.read_csv('train.csv', sep=',')
data_uji = pd.read_csv('data_uji.csv', sep=',')

In [ ]:
data_mentah.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title,Family_Size
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,Mr,1
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,Mrs,1
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,Miss,0
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,Mrs,1
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,Mr,0


In [ ]:
data_uji.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title,Family_Size
0,34.5,NaN,Q,7.8292,"Kelly, Mr. James",0,892,3,male,0,NaN,330911,Mr,0
1,47.0,NaN,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,893,3,female,1,NaN,363272,Mrs,1
2,62.0,NaN,Q,9.6875,"Myles, Mr. Thomas Francis",0,894,2,male,0,NaN,240276,Mr,0
3,27.0,NaN,S,8.6625,"Wirz, Mr. Albert",0,895,3,male,0,NaN,315154,Mr,0
4,22.0,NaN,S,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,896,3,female,1,NaN,3101298,Mrs,2


3. Data Preparation


In [ ]:
print(data_mentah.isna().sum())
data_mentah.describe()


Age              0
Cabin          687
Embarked         0
Fare             0
Name             0
Parch            0
PassengerId      0
Pclass           0
Sex              0
SibSp            0
Survived         0
Ticket           0
Title            0
Family_Size      0
dtype: int64


,Age,Fare,Parch,PassengerId,Pclass,SibSp,Survived,Family_Size
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,29.445196,32.204208,0.381594,446.000000,2.308642,0.523008,0.383838,0.904602
std,13.244896,49.693429,0.806057,257.353842,0.836071,1.102743,0.486592,1.613459
min,0.420000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,22.000000,7.910400,0.000000,223.500000,2.000000,0.000000,0.000000,0.000000
50%,30.000000,14.454200,0.000000,446.000000,3.000000,0.000000,0.000000,0.000000
75%,35.500000,31.000000,0.000000,668.500000,3.000000,1.000000,1.000000,1.000000
max,80.000000,512.329200,6.000000,891.000000,3.000000,8.000000,1.000000,10.000000


In [ ]:
print(data_uji.isna().sum())
data_uji.describe()

Age             0
Cabin          10
Embarked        0
Fare            0
Name            0
Parch           0
PassengerId     0
Pclass          0
Sex             0
SibSp           0
Survived       10
Ticket          0
Title           0
Family_Size     0
dtype: int64


,Age,Cabin,Fare,Parch,PassengerId,Pclass,SibSp,Survived,Family_Size
count,10.000000,0.0,10.000000,10.000000,10.00000,10.000000,10.000000,0.0,10.000000
mean,30.150000,NaN,12.270010,0.200000,896.50000,2.800000,0.500000,NaN,0.700000
std,14.541225,NaN,7.777627,0.421637,3.02765,0.421637,0.707107,NaN,0.948683
min,14.000000,NaN,7.000000,0.000000,892.00000,2.000000,0.000000,NaN,0.000000
25%,21.250000,NaN,7.679200,0.000000,894.25000,3.000000,0.000000,NaN,0.000000
50%,26.500000,NaN,8.943750,0.000000,896.50000,3.000000,0.000000,NaN,0.000000
75%,33.375000,NaN,11.637500,0.000000,898.75000,3.000000,1.000000,NaN,1.750000
max,62.000000,NaN,29.000000,1.000000,901.00000,3.000000,2.000000,NaN,2.000000


In [ ]:
#drop kolom yang tidak dibutuhkan dari kedua dataframe
ft = ['Name', 'Ticket', 'Title']
data_mentah = data_mentah.drop('Cabin',1)
data_uji = data_uji.drop('Cabin',1)
data_mentah = data_mentah.drop(ft, 1)
data_uji = data_uji.drop(ft, 1)

In [ ]:
data_mentah.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Family_Size
0,22.0,S,7.2500,0,1,3,male,1,0.0,1
1,38.0,C,71.2833,0,2,1,female,1,1.0,1
2,26.0,S,7.9250,0,3,3,female,0,1.0,0
3,35.0,S,53.1000,0,4,1,female,1,1.0,1
4,35.0,S,8.0500,0,5,3,male,0,0.0,0


In [ ]:
data_uji.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Family_Size
0,34.5,Q,7.8292,0,892,3,male,0,NaN,0
1,47.0,S,7.0000,0,893,3,female,1,NaN,1
2,62.0,Q,9.6875,0,894,2,male,0,NaN,0
3,27.0,S,8.6625,0,895,3,male,0,NaN,0
4,22.0,S,12.2875,1,896,3,female,1,NaN,2


In [ ]:
replace1 = ['male','female']
replacement1 = [0, 1]
replace2 = ['S', 'C', 'Q']
replacement2 = [1, 2, 3]

data_mentah['Sex'].replace(replace1, replacement1, inplace = True)
data_uji['Sex'].replace(replace1, replacement1, inplace = True)
data_mentah['Embarked'].replace(replace2, replacement2, inplace = True)
data_uji['Embarked'].replace(replace2, replacement2, inplace = True)

data_mentah.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Family_Size
0,22.0,1,7.2500,0,1,3,0,1,0.0,1
1,38.0,2,71.2833,0,2,1,1,1,1.0,1
2,26.0,1,7.9250,0,3,3,1,0,1.0,0
3,35.0,1,53.1000,0,4,1,1,1,1.0,1
4,35.0,1,8.0500,0,5,3,0,0,0.0,0


In [ ]:
X = data_mentah.copy().drop('Survived',1)
y = data_mentah['Survived']

In [ ]:
model_lr = LogisticRegression()
rfe = fs.RFE(model_lr)
rfe.fit(X,y)
print(f'Support = {rfe.support_}')
print(f'Ranking = {rfe.ranking_}')

Support = [False  True False False False  True  True  True False]
Ranking = [3 1 4 6 5 1 1 1 2]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [ ]:
X = data_mentah[['Embarked', 'Pclass', 'Sex', 'SibSp']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
      test_size = 0.2, random_state = 40)

print(f'dimensi dari X_train : {len(X_train)}')
print(f'dimensi dari y_train : {len(y_train)}')
print(f'dimensi dari X_test : {len(X_test)}')
print(f'dimensi dari y_test : {len(y_test)}')

dimensi dari X_train : 712
dimensi dari y_train : 712
dimensi dari X_test : 179
dimensi dari y_test : 179


4. Data Processing

In [ ]:
#Logistic Regression
lg = LinearRegression()
lg.fit(X_train,y_train)

LinearRegression()

In [ ]:
y_predict = lg.predict(X_test)
auc = metrics.accuracy_score(y_test, y_predict.round())

print(f'Nilai AUC : {auc}')

Nilai AUC : 0.8156424581005587


In [ ]:
X_uji = data_uji[['Embarked', 'Pclass', 'Sex', 'SibSp']]
y_pred = lg.predict(X_uji)
data_uji = data_uji.drop('Survived', 1)
data_uji['Survived'] = y_pred.round()
data_uji = data_uji[['Age', 'Embarked', 'Fare', 'Parch',  'PassengerId', 'Pclass', 'Sex', 'SibSp', 'Survived','Family_Size',]]
data_uji

,Age,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Family_Size
0,34.5,3,7.8292,0,892,3,0,0,0.0,0
1,47.0,1,7.0000,0,893,3,1,1,1.0,1
2,62.0,3,9.6875,0,894,2,0,0,0.0,0
3,27.0,1,8.6625,0,895,3,0,0,0.0,0
4,22.0,1,12.2875,1,896,3,1,1,1.0,2
5,14.0,1,9.2250,0,897,3,0,0,0.0,0
6,30.0,3,7.6292,0,898,3,1,0,1.0,0
7,26.0,1,29.0000,1,899,2,0,1,0.0,2
8,18.0,2,7.2292,0,900,3,1,0,1.0,0
9,21.0,1,24.1500,0,901,3,0,2,0.0,2


In [ ]:
insight:
Fitur yang berpengaruh signifikan terhadap selamat atau tidaknya penumpang adalah fitur Embarked, Pclass, Sex, dan SibSp. 
Berdasarkan lg.coef_ diperoleh nilai parameter dari masing masing fitur berturut-turut adalah ( 0.04866096, -0.15681182,  0.50377491, -0.02321819).
Interpretasi dari model yang terbentuk dapat dilihat dari nilai exponensial dari parameter betha atau odds ratio masing-masing fitur. 
Nilai koefisien beta pada fitur Embarked menunjukkan bahwa apabila peningkatan satu satuan Embarkasi akan meningkatkan kemungkinan penumpang selamat sebesar 1.05 kali, yang artinya penumpang dari embarkasi Q memiliki kemungkinan selamat lebih besar. 
Apabila terjadi peningkatan satu satuan dari PClass (kelas tiket penumpang) akan meningkatkan resiko penumpang tidak selamat sebesar (1/exp(beta)) atau 1.17 kali yang artinya kemungkinan penumpang first class (Pclass = 1) akan selamat lebih besar. 
Perempuan memiliki kemungkinan selamat 1.65 kali lebih besar dibandingkan laki-laki. Sedangkan adanya peningkatan jumlah saudara/pasangan (SibSp) sebesar satu satuan akan meningkatkan resiko penumpang tidak selamat sebesar 1.02 kali.
Pada hasil klasifikasi data_uji dapat diketahui bahwa hanya terdapat 4 penumpang yang masuk kedalam kelompok 1 (selamat) dan 6 penumpang tidak selamat. 
Semua penumpang selamat pada data_uji berjenis kelamin perempuan, memiliki satu saudara/pasangan, serta berada dalam third class. 
Nilai AUC model sebesar 0.815 menunjukkan bahwa model memiliki tingkat akurasi tinggi karena berada di rentang nilai AUC 0.8-0.9 sehingga model termasuk dalam kategori good classification.